<h2>OVERVIEW</h2>
<p>The first part of the question asks the candidate to compare simulated values of stock price at maturity, using the Closed Form solution, Euler Maruyama Scheme, and Milstein Scheme. </p>
<p>The second part of the question asks the candidate to build an Asian option pricer for calls and puts, taking into account the variation of input parameters.</p>
<p>The third part of the question asks the candidate to compare the value of the Asian option whilst varying the input parameters.</p>

<h3>Importing Libraries</h3>

In [1]:
# Importing libraries
import pandas as pd
from numpy import *
import matplotlib.pyplot as plt

# Set max row to 300
pd.set_option('display.max_rows', 300)

<h2>PART 1: Comparing Closed Form Solution, EM Scheme, MS Scheme.</h2>
<p>First we defining Functions for the Closed Form solution, E-M Scheme and Milstein Scheme</p>

<p>(1) Closed Form Solution</p>

In [2]:
# define simulation function
def simulate_path_CF(s0, mu, sigma, horizon, n_sims):

    # set random seed for reproducibility
    random.seed(1000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    sigma = sigma           # volatility
    T = horizon             # time horizon
    n = n_sims              # number of simulation
    
    # simulate 'n' asset price path with '2' time points - 0 and T
    S = zeros((2, n))
    S[0] = S0
    
    w = random.standard_normal(n)
    S[1] = S0*exp((r-sigma**2/2)*T+sigma*(sqrt(T)*w))

    return S

<p>(2) Euler Maruyama Scheme</p>

In [3]:
# define simulation function
def simulate_path_EM(s0, mu, sigma, horizon, timesteps, n_sims):

    # set random seed for reproducibility
    random.seed(1000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    vol = sigma           # volatility
    T = horizon             # time horizon
    t = timesteps           # number of time steps
    n = n_sims              # number of simulation

    # define dt
    dt = T/t                # length of time interval

    # simulate 'n' asset price path with 't' timesteps
    S = zeros((t+1,n))
    S[0] = S0

    for i in range(0, t):
        w = random.standard_normal(n)
        S[i+1] = S[i] * (1 + r*dt + vol*sqrt(dt)*w)

    return S

<p>(3) Milstein Scheme</p>

In [4]:
# define simulation function
def simulate_path_MS(s0, mu, sigma, horizon, timesteps, n_sims):

    # set random seed for reproducibility
    random.seed(1000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    vol = sigma           # volatility
    T = horizon             # time horizon
    t = timesteps           # number of time steps
    n = n_sims              # number of simulation

    # define dt
    dt = T/t                # length of time interval

    # simulate 'n' asset price path with 't' timesteps
    S = zeros((t+1,n))
    S[0] = S0

    for i in range(0, t):
        w = random.standard_normal(n)
        S[i+1] = S[i] * (1 + r*dt + vol*sqrt(dt)*w + vol**2*(w**2-1)*dt/2)

    return S

In [5]:
## Assign simulated price path to check function is working as expected
price_path_CF = pd.DataFrame(simulate_path_CF(100,0.05,0.2,1,1000))
price_path_EM = pd.DataFrame(simulate_path_EM(100,0.05,0.2,1,20,1000))
price_path_MS = pd.DataFrame(simulate_path_MS(100,0.05,0.2,1,20,1000))

In [6]:
price_path_CF.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000
1,87.731282,109.876444,102.52161,117.217942,97.029092,111.393068,100.854887,93.613403,116.068123,93.900591,...,93.910128,88.322997,85.924861,91.643671,162.618455,104.263928,102.600662,106.995737,110.157367,102.33269


In [7]:
price_path_EM.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,96.652353,101.685250,100.136037,103.131504,98.904796,101.991783,99.769526,98.103450,102.911080,98.171944,...,98.174215,96.802661,96.187132,97.627936,110.451758,100.512856,100.153272,101.091182,101.742347,100.094794
2,96.735080,96.937133,106.083085,101.704968,105.510414,103.904861,100.532006,90.221314,108.549984,101.142612,...,95.783482,101.864015,91.454092,96.136737,110.441668,97.828900,102.343786,91.842892,100.688599,95.744417
3,99.109860,96.271501,96.986034,106.501537,96.621627,99.305448,111.759193,90.806013,109.941224,102.017823,...,97.431506,105.500205,81.499057,95.785747,108.923219,98.751328,106.761895,87.667108,95.538739,96.942207
4,106.312144,98.657092,103.648377,107.572122,100.293693,97.639413,104.817255,88.631427,111.060974,108.898408,...,96.465893,106.704146,84.124044,102.562166,102.029805,98.309128,111.493757,84.761371,95.155545,103.914347


In [8]:
price_path_MS.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,96.617068,101.595549,100.036102,103.073019,98.813844,101.906952,99.670680,98.026489,102.846486,98.093535,...,98.095759,96.762082,96.169666,97.562312,110.872138,100.413201,100.053319,100.994720,101.653482,99.994915
2,96.603278,96.872960,106.039059,101.557966,105.518768,103.729998,100.334052,90.388903,108.519630,101.001533,...,95.644206,101.844468,91.469874,95.989965,110.751509,97.674309,102.160367,92.100309,100.507403,95.654506
3,98.901702,96.115146,97.252701,106.347331,96.920761,99.148026,112.036777,90.885015,109.807737,101.776435,...,97.204531,105.434232,81.988740,95.545336,109.132669,98.499950,106.552883,87.926672,95.410931,96.760309
4,106.233444,98.424602,104.049172,107.313052,100.568363,97.403906,105.199444,88.649438,110.819562,108.753600,...,96.151450,106.536173,84.583692,102.431701,102.353025,97.962774,111.262112,84.980264,94.934861,103.855762


<p>Next, we compare resultsbetween the CF Solution, EM Scheme and MS Scheme</p>
<p>To do this, we define a function that takes in the asset data, simulation parameters, the variable to vary (timesteps number of simulations), and the list of values for comparison.</p>
<p>The function outputs a dataframe of the asset value at maturity and volatility at various values of the comparison variable (timesteps number of simulations).</p>

In [9]:
def compare_schemes(s0, mu, sigma, horizon, compare, val_list, timesteps=20, n_sims=1000):
    
    
    '''
    Compares the price and volatility at maturity between 
    (1) The Closed Form Solution, 
    (2) Euler Maruyama Scheme, 
    (3) MilStein Scheme.
    
    Parameters:
    s0:        Initial spot price,
    mu:        mu = rf in risk neutral framework,
    sigma:     volatility of the underlying asset,
    horizon:   Time horizon,
    
    compare:   The variable to compare, takes in a string value - 
               "n" for number of simulations,
                "t" for number of time steps
                
    val_list:  The list of values to compare.
    
    timesteps: Number of time steps - default value = 20,
    n_sims:    Number of simulation - default value = 1000,
    
    Output:
    Outputs a dataframe that stores in each column:
    
    (1) The list of values assigned to the comparison variable, 
        i.e., values in the input val_list
    (2) The price at time T simulated using the Closed Form solution
    (3) The volatility at time T simulated using the Closed Form solution
    (4) The price at time T simulated using the Euler Maruyama Scheme
    (5) The volatility at time T simulated using the Euler Maruyama Scheme
    (6) The price at time T simulated using the Milstein Scheme
    (7) The volatility at time T simulated using the Milstein Scheme
    
    The function also plots two graphs: 
    (1) Comparing the calculated price at time T using 
        the three schemes vs the actual (theoretical) price at time T
    (2) Comparing the calculated volatility using 
        the three schemes vs the actual (theoretical) price at time T
    
    '''
    
    # set random seed for reproducibility
    random.seed(10000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    vol = sigma             # volatility
    T = horizon             # time horizon
    
    if compare.lower() != "t" and compare.lower() != "n":
        return "Invalid input for the 'compare' parameter."
    
    # Comparing results whilst varying timesteps "t"
    if compare.lower() == "t":
        
        # Calculating actual forward price at maturity & actual volatility
        act_St = [S0*exp(r*T) for timestep in val_list]
        act_vol = [sigma for timestep in val_list]
        
        # Calculating forward price at maturity & actual volatility using 
        # closed form solution
        CF_St = pd.DataFrame(simulate_path_CF(S0,r,vol,T,n_sims)).iloc[-1].mean()
        CF_vol = log(simulate_path_CF(S0,r,vol,T,n_sims)[-1]/S0).std()
        
        # Calculating forward price at maturity & actual volatility using 
        # Euler Maruyama Scheme
        EM_St = [pd.DataFrame(simulate_path_EM(S0,r,vol,T,timestep,n_sims)).iloc[-1].mean() 
                        for timestep in val_list]
        EM_vol = [log(simulate_path_EM(S0,r,vol,T,timestep,n_sims)[-1]/S0).std() 
                        for timestep in val_list]
        
        # Calculating forward price at maturity & actual volatility using 
        # Milstein Scheme
        MS_St = [pd.DataFrame(simulate_path_MS(S0,r,vol,T,timestep,n_sims)).iloc[-1].mean()
                        for timestep in val_list]
        MS_vol = [log(simulate_path_EM(S0,r,vol,T,timestep,n_sims)[-1]/S0).std()
                        for timestep in val_list]
    
    # Comparing results whilst varying number of simulations "n"
    if compare.lower() == "n":
        
        # Calculating actual forward price at maturity & actual volatility
        act_St = [S0*exp(r*T) for n in val_list]
        act_vol = [sigma for n in val_list]
        
        # Calculating forward price at maturity & actual volatility using 
        # closed form solution
        CF_St = [pd.DataFrame(simulate_path_CF(S0,r,vol,T,n)).iloc[-1].mean()
                             for n in val_list]
        CF_vol = [log(simulate_path_CF(S0,r,vol,T,n)[-1]/S0).std()
                             for n in val_list]
        
        # Calculating forward price at maturity & actual volatility using 
        # Euler Maruyama Scheme
        EM_St = [pd.DataFrame(simulate_path_EM(S0,r,vol,T,timesteps,n)).iloc[-1].mean()
                             for n in val_list]
        EM_vol = [log(simulate_path_EM(S0,r,vol,T,timesteps,n)[-1]/S0).std()
                             for n in val_list]
        
        # Calculating forward price at maturity & actual volatility using Milstein Scheme
        MS_St = [pd.DataFrame(simulate_path_MS(S0,r,vol,T,timesteps,n)).iloc[-1].mean()
                             for n in val_list]
        MS_vol = [log(simulate_path_MS(S0,r,vol,T,timesteps,n)[-1]/S0).std()
                             for n in val_list]
        
    results = pd.DataFrame.from_dict({
        f"Values of '{compare.lower()}'": val_list,
        "Actual Price at T": act_St,
        "Actual Sigma at T": act_vol,
        "CF Price at T": CF_St,
        "CF Sigma at T": CF_vol,
        "EM Price at T": EM_St,
        "EM Sigma at T": EM_vol,
        "MS Price at T": MS_St,
        "MS Sigma at T": MS_vol,
    })
    
    # Actual price and sigma at time T
    ax1 = results.plot(x=f"Values of '{compare.lower()}'", y='Actual Price at T')
    results.plot(ax=ax1, x=f"Values of '{compare.lower()}'", y='CF Price at T')
    results.plot(ax=ax1, x=f"Values of '{compare.lower()}'", y='EM Price at T')
    results.plot(ax=ax1, x=f"Values of '{compare.lower()}'", y='MS Price at T')
    ax1.set_title(f"Simulated St at varying '{compare.lower()}' values'")
    box1 = ax1.get_position()
    ax1.set_position([box1.x0, box1.y0, box1.width * 1.5, box1.height]);
    
    ax2 = results.plot(x=f"Values of '{compare.lower()}'", y='Actual Sigma at T')
    results.plot(ax=ax2, x=f"Values of '{compare.lower()}'", y='CF Sigma at T')
    results.plot(ax=ax2, x=f"Values of '{compare.lower()}'", y='EM Sigma at T')
    results.plot(ax=ax2, x=f"Values of '{compare.lower()}'", y='MS Sigma at T')
    ax2.set_title(f"Simulated Sigma at varying '{compare.lower()}' values'")
    box2 = ax2.get_position()
    ax2.set_position([box2.x0, box2.y0, box2.width * 1.5, box2.height]);
    
    return results

<h3>Comparing the three schemes by varying MC Simulation parameters</h3>
<h4>(1) Varying timesteps</h4>

In [ ]:
# Vary the timesteps to see the effect on simulated forward price

compare_timestep = compare_schemes(100, 0.05, 0.2, 1, "t", 
                list(range(10, 200, 2)),
                timesteps=20, n_sims=10000)

<h5>Comment: </h5>
<p>We can see that generally the forward stock price and sigma from all three simulations are reasonably close to the actual values. As 't' becomes larger, the forward price tends toward the actual result and the volatility fluctuates around the actual result. In both cases, the error is satisfactorily small.</p>
<p>The Euler Maruyama Scheme and the Milstein Scheme have similar values. For the forward price, we see that they quite clearly converge as timesteps increase, for the volatility, the values seem to be very close throughout.</p>
<p>We can informally conclude that as the time step increases, the subsequent results from the simulated paths become more accurate.</p>

<h4>(2) Varying the number of simulations</h4>

In [ ]:
# Vary the number of simulations to see the effect on simulated forward price

comepare_n = compare_schemes(100, 0.05, 0.2, 1, "n", 
                [100, 1000, 2000, 3000, 5000, 10000, 50000, 
                 100000, 200000, 500000, 700000, 1000000],
                timesteps=52, n_sims=10000)

<h4>Comments: </h4>
<p>Similar to above, generally the forward stock price and sigma from all three simulations converge to the actual value as the number of simulations increase.</p>
<p>The results from the EM Scheme and the Milstein Scheme are very close and follow the same trends. </p>
<p>We can conclude that as the number of simulations increase, the subsequent result becomes more accurate.</p>

<h2>PART 2: Asian Option Pricer</h2>

<h3>Asian Option Pricer Function</h3>

<p>First we create an Asian Option pricer function.<p>

In [ ]:
def price_asian(s0=100, mu=0.05, sigma=0.2, horizon=1, 
                timesteps=252, n_sims=10000, strike=100,
                OpType="C", StrikeType="fixed", 
                SampleType="C", AvgType="A", AvgStep=-1):

    '''
    Prices an Asian option with the specifications given in parameter in puts.
    
    Parameters:
    s0:           Initial spot price - default value = 100
    mu:           mu = rf in risk neutral framework - default value = 0.05
    sigma:        volatility of the underlying asset - default value = 0.2
    horizon:      Time horizon - default value = 9
    timesteps:    Number of time steps - default value = 20,
    n_sims:       Number of simulation - default value = 1000,
    strike:       Strike of the option (at fixed strike) - default value = 100,
    
    OpType:       Type of the option, takes one of two values, case insensitive:
                  (1) "C" for call, 
                  (2) "P" for put.
              
    StrikeType:   Type of the strike, takes one of two values, case insensitive:
                  (1) "Fixed" for fixed strike, 
                  (2) "Floating" for floating strike.
    
    SampleType:   Type of sampling method, takes one of two values, case insensitive:
                  (1) "D" for discrete sampling, 
                  (2) "C" for continuous sampling.
    
    AvgType:      Type of averaging method used, takes one of two values, case insensitive:
                  (1) "A" for arithmetic average, 
                  (2) "G" for geometric average.
    
    AvgStep:      If SampleType = "D" (discrete sampling), specifies the step \
                  difference between each sampled point.
                  e.g. AvgStep = 3, then the sample is S(t), S(t-3), S(t-6)...
     
    Output:       Output the price of the option specified.

                
    
    '''
    
    # implementing checks
    
    if not OpType.lower() in ['c','p']:
        return "Invalid option type - please choose between 'C' or 'P'."
    
    if not StrikeType.lower() in ['fixed','floating']:
        return "Invalid strike type - please choose between 'fixed' or 'floating'."
    
    if not SampleType.lower() in ['d','c']:
        return "Invalid sampling method - please choose between 'D' or 'C'."
    
    if not AvgType.lower() in ['a','g']:
        return "Invalid option type - please choose between 'A' or 'G'."
    
    # set random seed for reproducibility
    random.seed(1000)

    # read parameters
    S0 = s0                 # initial spot price
    r = mu                  # mu = rf in risk neutral framework
    vol = sigma             # volatility
    T = horizon             # time horizon
    t = timesteps           # number of time steps
    n = n_sims              # number of simulation
    K = strike              # the Strike price
    
    
    # simulate asset path using the Milstein Scheme path generation function defined
    SPath = simulate_path_MS(S0,r,vol,T,t,n)
    
    # calculating averages in accordance with SampleType & AvgType & AvgStep
    if SampleType.lower() == "c":
        
        if AvgType.lower() == "a":
            avg = mean(SPath, axis=0)
        elif AvgType.lower() == "g":
            avg = exp(mean(log(SPath), axis=0))
            
    elif SampleType.lower() == "d":
        
        if AvgType.lower() == "a":
            avg = mean(SPath[list(range(t,-1,AvgStep))], axis=0)
        elif AvgType.lower() == "g":
            avg = exp(mean(log(SPath[list(range(t,-1,AvgStep))]), axis=0))
        
    # replacing S or K with the calculated average, in accordance with StrikeType
    if StrikeType.lower() == 'fixed':
        S = avg
        
    elif StrikeType.lower() == 'floating':
        S = SPath[-1]
        K = avg
    
    # calculating discounted value of the expected payoff
    
    C0 = exp(-r*T) * mean(maximum(0, S-K))
    P0 = exp(-r*T) * mean(maximum(0, K-S))
    
    # output price depending on the OpType
    
    if OpType.lower() == "c":
        return C0
    
    if OpType.lower() == "p":
        return P0

<h2>PART 3: Asian Option Price at Varying Parameters</h2>

<h3>(1) Varying the Spot S0</h3>
<p>Holding fixed: sigma=0.2, mu=0.05, horizon=1, timesteps=252, strike=100, SampleType='C', AvgType='A'</p>

In [ ]:
# Values of S0
S0_vals = list(range(50, 150, 10))

# Fixed Strike Call
S0_FX_C = [price_asian(s0=val, OpType="C", StrikeType="fixed") for val in S0_vals]

# Floating Strike Call
S0_FL_C = [price_asian(s0=val, OpType="C", StrikeType="floating") for val in S0_vals]

# Fixed Strike Put
S0_FX_P = [price_asian(s0=val, OpType="P", StrikeType="fixed") for val in S0_vals]

# Floating Strike Put
S0_FL_P = [price_asian(s0=val, OpType="P", StrikeType="floating") for val in S0_vals]

# Create dataframe for varying S0 values and price calculated at each S0
S0_df = pd.DataFrame.from_dict({
    'S0 Values': S0_vals,
    'Fixed Strike Call Price': S0_FX_C,
    'Floating Strike Call Price': S0_FL_C,
    'Fixed Strike Put Price': S0_FX_P,
    'Floating Strike Put Price': S0_FL_P,
})

# Plot results
ax = S0_df.plot(x='S0 Values', y='Fixed Strike Call Price');
S0_df.plot(ax=ax, x='S0 Values', y='Floating Strike Call Price');
S0_df.plot(ax=ax, x='S0 Values', y='Fixed Strike Put Price');
S0_df.plot(ax=ax, x='S0 Values', y='Floating Strike Put Price');
ax.set_title("Asian Option Price at Varying Spots");
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 1.5, box.height]);

<h3>Interpretation</h3>
<h4>Fixed Strike Call</h4>
<p><u>As the Spot at time 0 increases, the option price increases. </u>Because a larger starting pushes up the end value at maturity, hence increasing the value of S - K. Moreover, larger values of St gives a larger diffusion of the geometric Brownian Motion, which also contributes to a larger expected value of ST, further increasing the call payoff.</p>
<h4>Fixed Strike Put</h4>
<p><u>As the Spot at time 0 increases, the option price decreases. </u>Because a larger starting pushes up the end value at maturity, hence decreasing the value of K - S. Moreover, larger values of St gives a larger diffusion of the geometric Brownian Motion, which also contributes to a larger expected value of ST, further decreasing the put payoff.</p>
<h4>Floating Strike Call</h4>
<p><u>As the Spot at time 0 increases, the option price slowly increases. </u>The effects of the previous two products is heavily diminished as now S and K move together. However, we can still see a slower increase in the option price as a larger S0 still gives a larger diffusion to the geometric Brownian Motion, which contributes to a larger average of ST. we can still see a slower increase in the option price as a larger S0 still gives a larger <u>diffusion</u> to the geometric Brownian Motion, meaning that at maturity the distribution of ST becomes more widely dispersed, producing a larger expected payoffs S-K.</p>
<h4>Floating Strike Put</h4>
<p><u>As the Spot at time 0 increases, the option price slowly increases. </u>The effects of the previous two products is heavily diminished as now S and K move together. However, we can still see a slower increase in the option price as a larger S0 still gives a larger diffusion to the geometric Brownian Motion, which contributes to a larger average of ST. we can still see a slower increase in the option price as a larger S0 still gives a larger <u>diffusion</u> to the geometric Brownian Motion, meaning that at maturity the distribution of ST becomes more widely dispersed, producing a larger expected payoffs K-S.</p>

<h3>(2) Varying the Volatility</h3>
<p>Holding fixed: S0=100, mu=0.05, horizon=1, timesteps=252, strike=100, SampleType='C', AvgType='A'</p>

In [ ]:
# Values of Volatility (sigma)
vol_vals = list(arange(0, 0.5, 0.02))

# Fixed Strike Call
vol_FX_C = [price_asian(sigma=val, OpType="C", StrikeType="fixed") for val in vol_vals]

# Floating Strike Call
vol_FL_C = [price_asian(sigma=val, OpType="C", StrikeType="floating") for val in vol_vals]

# Fixed Strike Put
vol_FX_P = [price_asian(sigma=val, OpType="P", StrikeType="fixed") for val in vol_vals]

# Floating Strike Put
vol_FL_P = [price_asian(sigma=val, OpType="P", StrikeType="floating") for val in vol_vals]

# Create dataframe for varying volatility values and price calculated at each S0
vol_df = pd.DataFrame.from_dict({
    'Volatility Values': vol_vals,
    'Fixed Strike Call Price': vol_FX_C,
    'Floating Strike Call Price': vol_FL_C,
    'Fixed Strike Put Price': vol_FX_P,
    'Floating Strike Put Price': vol_FL_P,
})

# Plot results
ax = vol_df.plot(x='Volatility Values', y='Fixed Strike Call Price');
vol_df.plot(ax=ax, x='Volatility Values', y='Floating Strike Call Price');
vol_df.plot(ax=ax, x='Volatility Values', y='Fixed Strike Put Price');
vol_df.plot(ax=ax, x='Volatility Values', y='Floating Strike Put Price');
ax.set_title("Asian Option Price at Varying Volatilities");
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 1.5, box.height]);

<h3>Interpretation</h3>
<h4>For all four product categories:</h4>
<p><u>As the volatility increases, the option price increases. </u>Options are generally long volatility products since the payoff of the option depends on how much the asset moves. i.e., the wider the diffusion, the more likely that the final S-K (for calls) and K-S (for puts) can have larger values.</p>


<h3>(3) Varying the Risk Free rate</h3>
<p>Holding fixed: S0=100, sigma=0.2, horizon=1, timesteps=252, strike=100, SampleType='C', AvgType='A'</p>

In [ ]:
# Values of Risk Free Interest Rate (mu)
r_vals = list(arange(0, 0.1, 0.01))

# Fixed Strike Call
r_FX_C = [price_asian(mu=val, OpType="C", StrikeType="fixed") for val in r_vals]

# Floating Strike Call
r_FL_C = [price_asian(mu=val, OpType="C", StrikeType="floating") for val in r_vals]

# Fixed Strike Put
r_FX_P = [price_asian(mu=val, OpType="P", StrikeType="fixed") for val in r_vals]

# Floating Strike Put
r_FL_P = [price_asian(mu=val, OpType="P", StrikeType="floating") for val in r_vals]

# Create dataframe for varying risk free rate values and price calculated at each S0
r_df = pd.DataFrame.from_dict({
    'Risk Free Rate Values': r_vals,
    'Fixed Strike Call Price': r_FX_C,
    'Floating Strike Call Price': r_FL_C,
    'Fixed Strike Put Price': r_FX_P,
    'Floating Strike Put Price': r_FL_P,
})

# Plot results
ax = r_df.plot(x='Risk Free Rate Values', y='Fixed Strike Call Price');
r_df.plot(ax=ax, x='Risk Free Rate Values', y='Floating Strike Call Price');
r_df.plot(ax=ax, x='Risk Free Rate Values', y='Fixed Strike Put Price');
r_df.plot(ax=ax, x='Risk Free Rate Values', y='Floating Strike Put Price');
ax.set_title("Asian Option Price at Varying Risk-Free Rates");
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 1.5, box.height]);

<h3>Interpretation</h3>
<h4>Fixed Strike Call</h4>
<p><u>As the Risk Free Rate increases, the option price increases. </u>Because a higher interest rate pushes up the forward price at time T. Thereby decreasing the payoff of the call.</p>
<h4>Fixed Strike Put</h4>
<p><u>As the Risk Free Rate increases, the option price decreases. </u>Because a higher interest rate pushes up the forward value at time T. Thereby decreasing the payoff of the put.</p>
<h4>Floating Strike Call</h4>
<p><u>As the Risk Free Rate increases, the option price increases. </u>A higher interest rate will on average push values of the underlying asset throughout the time horison, therefore the difference between a sampled average and the value at maturity (S - K) will become larger. This can be easily thought of using a simple example of the sampled asset values being S(t-4) = 10, S(t-2) = 20, S(t) = 30: initially the payoff is 30 - 20, where 20 is the arithmetic mean of the three samples. If interest increases and now each of these sampled point values become 11, 22, 33, then the arithmetic mean now becomes 22, which makes the payoff 33 - 22 = 11 > 10.</p>
<h4>Floating Strike Put</h4>
<p><u>As the Risk Free Rate increases, the option price decreases. </u>This can be inferred from the same logic as above for Floating Strike Calls.</p>

<h5><u>Note: </u>For all cases above the effect of discounting is less significant than the increase of forward values resulting from rising interest rates.</h5>

<h3>(4) Varying the Averaging method: Arithmetic vs Geometric</h3>
<p>Holding fixed: S0=100, sigma=0.2, mu = 0.05, horizon=1, timesteps=252, strike=100, SampleType='C'</p>

In [ ]:
# Values denoting different averaging methods
avg_vals = ['A', 'G']

# Fixed Strike Call
avg_FX_C = [price_asian(AvgType=val, OpType="C", StrikeType="fixed") for val in avg_vals]

# Floating Strike Call
avg_FL_C = [price_asian(AvgType=val, OpType="C", StrikeType="floating") for val in avg_vals]

# Fixed Strike Put
avg_FX_P = [price_asian(AvgType=val, OpType="P", StrikeType="fixed") for val in avg_vals]

# Floating Strike Put
avg_FL_P = [price_asian(AvgType=val, OpType="P", StrikeType="floating") for val in avg_vals]

# Create dataframe for different averaging methods and price calculated at each S0
avg_df = pd.DataFrame.from_dict({
    'Averaging Method': avg_vals,
    'Fixed Strike Call Price': avg_FX_C,
    'Floating Strike Call Price': avg_FL_C,
    'Fixed Strike Put Price': avg_FX_P,
    'Floating Strike Put Price': avg_FL_P,
})

# Plot results
ax = avg_df.plot.scatter(x='Averaging Method', y='Fixed Strike Call Price',
                         s=40, c='blue', grid=True, legend=True);
avg_df.plot.scatter(ax=ax, x='Averaging Method', y='Floating Strike Call Price',
                    s=40, c='green', grid=True, legend=True);
avg_df.plot.scatter(ax=ax, x='Averaging Method', y='Fixed Strike Put Price',
                    s=40, c='orange', grid=True, legend=True);
avg_df.plot.scatter(ax=ax, x='Averaging Method', y='Floating Strike Put Price',
                    s=40, c='black', grid=True, legend=True);
ax.set_title("Asian Option Price Using Different Averaging Methods");
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 1.5, box.height]);
ax.legend(['Fixed Strike Call Price','Floating Strike Call Price'
           ,'Fixed Strike Put Price','Floating Strike Put Price']
          ,loc='center left', bbox_to_anchor=(1, 0.5));


<h3>Interpretation</h3>
<h4>Fixed Strike Call</h4>
<p><u>The price is lower when taking geometric mean. </u>The geometric mean is always smaller than the arithmetic mean. Therefore the forward value is smaller when taking geometric mean, reducing the option price.</p>
<h4>Fixed Strike Put</h4>
<p><u>The price is higher when taking the geometric mean. </u>The geometric mean is always smaller than the arithmetic mean. Therefore the forward value is smaller when taking geometric mean, increasing the option price..</p>
<h4>Floating Strike Call</h4>
<p><u>The price is higher when taking geometric mean. </u>The geometric mean is always smaller than the arithmetic mean. Therefore the strike is smaller when taking geometric mean, increasing the option price.</p>
<h4>Floating Strike Put</h4>
<p><u>The price is lower when taking geometric mean. </u>The geometric mean is always smaller than the arithmetic mean. Therefore the strike is smaller when taking geometric mean, decreasing the option price.</p>

<h3>(5) Varying the Sampling method: Continuous vs Discrete (at various distance between each sample)</h3>
<p>Holding fixed: S0=100, sigma=0.2, mu = 0.05, horizon=1, timesteps=252, strike=100, AvgType='A'</p>

In [ ]:
# Values denoting different sampling methods
### Note since continuous sampling is equivalent to discrete sampling but with a step of -1, 
### We letavg_vals = 'D', and input a list of step lengths
### Values starting from -1, which denotes continuous sampling

step_vals = list(range(-1, -60, -1))

# Fixed Strike Call
step_FX_C = [price_asian(SampleType="D", AvgStep=val, OpType="C", StrikeType="fixed") 
             for val in step_vals]

# Floating Strike Call
step_FL_C = [price_asian(SampleType="D", AvgStep=val, OpType="C", StrikeType="floating") 
             for val in step_vals]

# Fixed Strike Put
step_FX_P = [price_asian(SampleType="D", AvgStep=val, OpType="P", StrikeType="fixed") 
             for val in step_vals]

# Floating Strike Put
step_FL_P = [price_asian(SampleType="D", AvgStep=val, OpType="P", StrikeType="floating") 
             for val in step_vals]

# Create dataframe for different averaging methods and price calculated at each S0
step_df = pd.DataFrame.from_dict({
    'Sampling Step (Starting from 1=Continuous)': [val*(-1) for val in step_vals],
    'Fixed Strike Call Price': step_FX_C,
    'Floating Strike Call Price': step_FL_C,
    'Fixed Strike Put Price': step_FX_P,
    'Floating Strike Put Price': step_FL_P,
})

# Plot results
ax = step_df.plot(x='Sampling Step (Starting from 1=Continuous)',
                  y='Fixed Strike Call Price');
step_df.plot(ax=ax, x='Sampling Step (Starting from 1=Continuous)',
             y='Floating Strike Call Price');
step_df.plot(ax=ax, x='Sampling Step (Starting from 1=Continuous)',
             y='Fixed Strike Put Price');
step_df.plot(ax=ax, x='Sampling Step (Starting from 1=Continuous)',
             y='Floating Strike Put Price');
ax.set_title("Asian Option Price Using Different Sampling Methods (from 1=Continuous)");
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 1.5, box.height]);

<h3>Interpretation</h3>
<p>At first glance, when we increase the sampling step from 0 to 60, the option price for all four products show a sawtooth shape. This is because as the discrete sampling step increases, every so often a "jump" occurs when the step crosses a point that decreases the sample size by 1. For example, suppose we are sampling backward from 100 to 0 out of 100 values, 100, 99, 98, ..., 0. When sample step = 21, 22, 23, 24, 25, their sample sizes all equal to 5. However, when the sampling step increases to 26, the sample size becomes 3 (100, 74, 48, 18). Therefore compared to when step=25, the sample "shifts" abruptly towards 100, thereby lifting the average and bumping up the price.</p>
<p>Another observation comes from comparing the many 'Peaks' on the saw tooth. We can see that the peak values slowly increase as well. Similar to the explanation of the jumps, at every new peak the sample size decreases by one and the sample shifts towards the point at maturity (larger values), and therefore the sample mean goes up with every new peak.</p>
<p>Aside from these "Jumps" and the resulting "Peaks" identified, we can also see that between each two jumps, the sample mean gradually decreases. Taking the same example as above, when step=21, sampled values = [100, 79, 58, 37, 16]; when step=22, sampled values = [100, 78, 56, 34, 12]; when step=22, sampled values = [100, 78, 56, 34, 12]; when step=23, sampled values = [100, 77, 54, 31, 8]. We can easily calculate that the sample mean at step=23 is smaller than at 22, which is smaller than at 21.</p>
<p>Because in all four products, the average replaces one of the forward (at maturity) or the strike values whilst the other one stays constant in our analysis, the payoff thereby driven by the sawtooth shaped change in the average. This can also explain why the price of the floating strike call and the fixed strike call (as with the floating strike put and fixed strike put) move in opposite directions, depending on whether the average replaces S or K.</p>


<h2>CONCLUSION</h2>
<h5>The follwing conclusions are drawn from the discussion above</h5>
<p>The Closed Form Solution, Euler Maruyama Scheme and Milstein Scheme are all valid schemes for simulating a stock's forward value. As the number of simulations increase, the values of from the EM Scheme converges very quickly to the Milstein Scheme (which we know adds a correction term to the EM Scheme) - which we have explored using both the sigma and the actual forward stock price. </p>
<p>We then varied parameters of the Asian Option (built on the simulated stock) and examined the impact on price for the fixed strike call, fixed strike put, floating strike call, floating strike call. From this we can see that the choices of parameters can have sizeable effects on the option price, mostly through impacting the sampled mean or the variance of the geometric Brownian Motion.</p>

<h2>Limitations</h2>
<p>An important limitation of the Asian Option pricer is the discrete sampling method. Here the method adopted is that we first define a sample step <em>n</em>. After that we the value at maturity as the first sample, and then take the values at t-n, t-2n, t-3n, ... until we go beyond t0. In real life, the contract of the option can take on various forms, and a different discrete sampling method may well be used. For example, we could instead choose a time period ( such as t-100 to t, then specify either a sample size <em>m</em> (from which we take <em>m</em> evenly spaced time points from the range (t-100, t), or, define a time step the same way as the pricer does, and keep sampling from t until we go beyond t0.</p>